In [11]:
import pandas as pd

## Prepare Co-Ocurrence Matrix

In [12]:
coocur_df = pd.read_excel('./00_raw_data/Nintendo_Cooccurrence_Matrix.xlsx')
coocur_df.rename(columns={'Unnamed: 0': 'Items'}, inplace=True)
coocur_df.head(5)

,Items,Nintendo Switch,Zelda: Breath of the Wild,Zelda: Tears of the Kingdom,Super Mario Odyssey,Mario Kart 8 Deluxe,Mario Party Superstars,Sonic Generations,Sonic Mania,Animal Crossing: New Horizons,Splatoon 3,Pikmin 4,Nintendo Switch Pro Controller,Joy-Con Controllers (Pair),Nintendo Switch Dock Set,Nintendo Switch Carrying Case,Nintendo Switch Screen Protector
0,Nintendo Switch,50000,30000,50000,60000,40000,20000,10000,10000,100000,30000,5000,50000,20000,10000,10000,5000
1,Zelda: Breath of the Wild,30000,100000,20000,15000,10000,5000,2000,3000,30000,20000,2000,5000,3000,2000,2000,1000
2,Zelda: Tears of the Kingdom,50000,20000,150000,25000,15000,10000,5000,5000,30000,20000,2000,10000,3000,2000,2000,1000
3,Super Mario Odyssey,60000,15000,25000,200000,30000,20000,5000,5000,25000,10000,2000,2000,1000,0,0,0
4,Mario Kart 8 Deluxe,40000,10000,15000,30000,150000,10000,2000,3000,13000,15000,1000,5000,3000,2000,1000,0


In [13]:
#-- Prepare Co-ocurrence matrix for final format
coocur_df_final = coocur_df.melt(id_vars='Items', var_name='product2', value_name='cooccurrence_count')
coocur_df_final.rename(columns={'Items': 'product1'}, inplace=True)
coocur_df_final.tail(10)

,product1,product2,cooccurrence_count
246,Sonic Generations,Nintendo Switch Screen Protector,1000
247,Sonic Mania,Nintendo Switch Screen Protector,1000
248,Animal Crossing: New Horizons,Nintendo Switch Screen Protector,10000
249,Splatoon 3,Nintendo Switch Screen Protector,5000
250,Pikmin 4,Nintendo Switch Screen Protector,0
251,Nintendo Switch Pro Controller,Nintendo Switch Screen Protector,0
252,Joy-Con Controllers (Pair),Nintendo Switch Screen Protector,0
253,Nintendo Switch Dock Set,Nintendo Switch Screen Protector,1000
254,Nintendo Switch Carrying Case,Nintendo Switch Screen Protector,0
255,Nintendo Switch Screen Protector,Nintendo Switch Screen Protector,10000


In [14]:
# remove self-cooccurrences
coocur_df_final = coocur_df_final[coocur_df_final['product1'] != coocur_df_final['product2']]

In [15]:
print('N rows before droping duplicates: ', coocur_df_final.shape)

# format dataframe so that product1 > product2 (alphabetically ordered) 
coocur_df_final[['product1', 'product2']] = coocur_df_final.apply(
    lambda row: sorted([row['product1'], row['product2']]), axis=1, result_type='expand'
)

# remove duplicate pairs (we only need one directional count (A, B), not (B, A))
coocur_df_final.drop_duplicates(ignore_index=True, keep='first', inplace=True)

print('N rows after droping duplicates: ', coocur_df_final.shape)

coocur_df_final.reset_index(drop=True, inplace=True)

coocur_df_final.head(10)

N rows before droping duplicates:  (240, 3)
N rows after droping duplicates:  (120, 3)


,product1,product2,cooccurrence_count
0,Nintendo Switch,Zelda: Breath of the Wild,30000
1,Nintendo Switch,Zelda: Tears of the Kingdom,50000
2,Nintendo Switch,Super Mario Odyssey,60000
3,Mario Kart 8 Deluxe,Nintendo Switch,40000
4,Mario Party Superstars,Nintendo Switch,20000
5,Nintendo Switch,Sonic Generations,10000
6,Nintendo Switch,Sonic Mania,10000
7,Animal Crossing: New Horizons,Nintendo Switch,100000
8,Nintendo Switch,Splatoon 3,30000
9,Nintendo Switch,Pikmin 4,5000


#### Data Verification

In [16]:
# verification
coocur_df_final[(coocur_df_final['product1'] == 'Nintendo Switch')|(coocur_df_final['product2'] == 'Nintendo Switch')]

,product1,product2,cooccurrence_count
0,Nintendo Switch,Zelda: Breath of the Wild,30000
1,Nintendo Switch,Zelda: Tears of the Kingdom,50000
2,Nintendo Switch,Super Mario Odyssey,60000
3,Mario Kart 8 Deluxe,Nintendo Switch,40000
4,Mario Party Superstars,Nintendo Switch,20000
5,Nintendo Switch,Sonic Generations,10000
6,Nintendo Switch,Sonic Mania,10000
7,Animal Crossing: New Horizons,Nintendo Switch,100000
8,Nintendo Switch,Splatoon 3,30000
9,Nintendo Switch,Pikmin 4,5000


### Save clean data to file

In [17]:
coocur_df_final.to_csv("./01_clean_data/coocurrences_data.csv", index=False)

## Prepare Products Dataset

In [1]:
import json

with open('./00_raw_data/dataset.json', 'r') as file:
    data_json = json.load(file)

data_json

{'Console': [{'name': 'Nintendo Switch',
   'Store A': 200000,
   'Store B': 150000,
   'Store C': 150000,
   'category': 'Console',
   'times_sold': 500000}],
 'Games': [{'name': 'Zelda: Breath of the Wild',
   'release_date': '2017-03-03',
   'times_sold': 250000,
   'Store A': 120000,
   'Store B': 80000,
   'Store C': 50000,
   'type': 'Adventure',
   'category': 'Game',
   'franchise': 'The Legend of Zelda',
   'min_age': 12},
  {'name': 'Zelda: Tears of the Kingdom',
   'release_date': '2023-05-12',
   'times_sold': 350000,
   'Store A': 200000,
   'Store B': 100000,
   'Store C': 50000,
   'type': 'Adventure',
   'category': 'Game',
   'franchise': 'The Legend of Zelda',
   'min_age': 12},
  {'name': 'Super Mario Odyssey',
   'release_date': '2017-10-27',
   'times_sold': 400000,
   'Store A': 150000,
   'Store B': 100000,
   'Store C': 150000,
   'type': 'Platformer',
   'category': 'Game',
   'franchise': 'Super Mario',
   'min_age': 7},
  {'name': 'Mario Kart 8 Deluxe',
   'r

In [2]:
#-- Create a flatten dictionary for final format
product_data = []

# fields to exclude from the text field
exclude_fields = {"times_sold", "Store A", "Store B", "Store C"}

for major_category, products in data_json.items():
    for product in products:
        product['major_category'] = major_category

        text_parts = [
            f"{k.upper()}: {v}" for k, v in product.items()
            if k not in exclude_fields and v is not None and k != 'text'
        ]
        product['text'] = '; '.join(text_parts)

        product_data.append(product)

product_data

[{'name': 'Nintendo Switch',
  'Store A': 200000,
  'Store B': 150000,
  'Store C': 150000,
  'category': 'Console',
  'times_sold': 500000,
  'major_category': 'Console',
  'text': 'NAME: Nintendo Switch; CATEGORY: Console; MAJOR_CATEGORY: Console'},
 {'name': 'Zelda: Breath of the Wild',
  'release_date': '2017-03-03',
  'times_sold': 250000,
  'Store A': 120000,
  'Store B': 80000,
  'Store C': 50000,
  'type': 'Adventure',
  'category': 'Game',
  'franchise': 'The Legend of Zelda',
  'min_age': 12,
  'major_category': 'Games',
  'text': 'NAME: Zelda: Breath of the Wild; RELEASE_DATE: 2017-03-03; TYPE: Adventure; CATEGORY: Game; FRANCHISE: The Legend of Zelda; MIN_AGE: 12; MAJOR_CATEGORY: Games'},
 {'name': 'Zelda: Tears of the Kingdom',
  'release_date': '2023-05-12',
  'times_sold': 350000,
  'Store A': 200000,
  'Store B': 100000,
  'Store C': 50000,
  'type': 'Adventure',
  'category': 'Game',
  'franchise': 'The Legend of Zelda',
  'min_age': 12,
  'major_category': 'Games',
  

### Save clean data to file

In [3]:
import json
with open("./01_clean_data/products_data.json", 'w', encoding='utf-8') as f:
    json.dump(product_data, f, ensure_ascii=False, indent=4)

### Test embeddings creation

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

for product in product_data:
    if "embedding" not in product.keys():
        client = OpenAI(api_key=os.environ["OPENAPI_KEY"])

        response = client.embeddings.create(
            input=product['text'],
            model=os.environ["OPENAI_EMB_MODEL"]
        )

        product["tokens"] = response.usage.total_tokens
        product["embedding"] = response.data[0].embedding

    break

In [9]:
product_data

[{'name': 'Nintendo Switch',
  'Store A': 200000,
  'Store B': 150000,
  'Store C': 150000,
  'category': 'Console',
  'times_sold': 500000,
  'major_category': 'Console',
  'text': 'NAME: Nintendo Switch; CATEGORY: Console; MAJOR_CATEGORY: Console',
  'tokens': 14,
  'embedding': [0.003805328393355012,
   0.010521418415009975,
   -0.03624226525425911,
   -0.013065598905086517,
   0.040903862565755844,
   -0.0028150128200650215,
   -0.04893581196665764,
   0.004160966724157333,
   0.016370298340916634,
   -0.017508340999484062,
   0.023898884654045105,
   0.039109256118535995,
   -0.017508340999484062,
   -0.0009321823599748313,
   0.010023524984717369,
   0.005304479971528053,
   -0.00986485555768013,
   0.02702850103378296,
   0.019423315301537514,
   0.005941892974078655,
   0.055019959807395935,
   0.034141264855861664,
   0.031099190935492516,
   0.015057173557579517,
   0.023176666349172592,
   0.058784253895282745,
   -0.04959237575531006,
   -0.03298133611679077,
   -0.00281501

In [10]:
import json
with open("./01_clean_data/products_data.json", 'w', encoding='utf-8') as f:
    json.dump(product_data, f, ensure_ascii=False, indent=4)